In [2]:
%pip install ipykernel
%pip install jupyter
%pip install torchvision
%pip install torch

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [70]:
# %pip uninstall 'numpy<2'
# %pip install 'numpy<2'

## imports


In [3]:
# from model import ASLCNN

# Import the necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torchvision import transforms


# Import project modules
from preprocess import get_data_loaders, get_transforms
from models.pretrained import get_pretrained_resnet
from models.model import ASLCNN
from train import train_model
from torchvision import datasets

import utils.model_utils as utils

# from utils.model_utils import load_model
# from utils.model_utils import save_model


# # Hyperparameters
import hyperparams as hp

In [4]:
%pwd
%ls

__pycache__/              dataset/                  preprocess.py
asl_model.pth             hyperparams.py            train.py
asl_model_custom.pth      models/                   utils/
asl_model_pretrained.pth  notebook.ipynb


# Load data


In [5]:
train_loader, val_loader = get_data_loaders()
train_loader2, val_loader2 = get_data_loaders()


#debugging
for images, labels in train_loader:
    print(f"Image batch shape: {images.shape}")  # [batch_size, channels, height, width]
    print(f"Label batch shape: {labels.shape}")  # [batch_size]
    break

Image batch shape: torch.Size([32, 3, 128, 128])
Label batch shape: torch.Size([32])


# Initialize model


### pretrained


In [8]:
num_classes = hp.NUM_CLASSES 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gotten from https://stackoverflow.com/questions/63302534/how-to-write-torch-devicecuda-if-torch-cuda-is-available-else-cpu-as-a-f

model = get_pretrained_resnet(num_classes)
model = model.to(device)

print(model)  #  print model structure


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### custom


In [9]:
num_classes = hp.NUM_CLASSES 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #gotten from https://stackoverflow.com/questions/63302534/how-to-write-torch-devicecuda-if-torch-cuda-is-available-else-cpu-as-a-f

custom_model = ASLCNN(num_classes ).to(device)
print(custom_model) #TODO ADD RELEVANT PARTS FOR LATER

ASLCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=262144, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=28, bias=True)
)


# Loss and optimizer


In [10]:
loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hp.LEARNING_RATE)


In [11]:
loss_fun2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(custom_model.parameters(), lr=hp.LEARNING_RATE)

In [12]:
# # example of iterating over one batch
# for images, labels in train_loader:
#     print(images.shape)  # check shape
#     print(labels)        # check label
#     break

# Training


### pretrained


In [13]:
# num_epochs = hp.NUM_EPOCHS
num_epochs = 13
trained_model = train_model(model, train_loader, val_loader, num_epochs, loss_fun, optimizer, device)


Epoch 1/13
--------------------


Training: 100%|██████████| 35/35 [00:17<00:00,  2.05it/s]


Train Loss: 3.4731 | Train Accuracy: 4.29%


Validating: 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


Validation Loss: 3.3785 | Validation Accuracy: 3.57%

Epoch 2/13
--------------------


...

Validation Loss: 1.9187 | Validation Accuracy: 53.57%

Epoch 13/13
--------------------


Training: 100%|██████████| 35/35 [00:16<00:00,  2.07it/s]


Train Loss: 2.1855 | Train Accuracy: 54.46%


Validating: 100%|██████████| 1/1 [00:00<00:00,  2.32it/s]

Validation Loss: 1.8364 | Validation Accuracy: 50.00%


## custom


In [7]:
# num_epochs = hp.NUM_EPOCHS
num_epochs = 5

custom_model = train_model(custom_model, train_loader2, val_loader2, num_epochs, loss_fun2, optimizer2, device)

NameError: name 'custom_model' is not defined

# saving


In [79]:
def save_model(model, path):
    '''
    '''
    torch.save(model.state_dict(), path)

def load_model(model_class, path, num_classes):
    '''
    '''
    model = model_class(num_classes)
    model.load_state_dict(torch.load(path))
    return model



In [80]:


save_model(trained_model, "asl_model_pretrained.pth")
print("model saved as 'asl_model_pretrained.pth'")
save_model(custom_model, "asl_model_custom.pth")
print("model saved as 'asl_model_custom.pth'")

model saved as 'asl_model_pretrained.pth'
model saved as 'asl_model_custom.pth'


# Testing


## pretrained


In [81]:
# eval  model
model.eval()  
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = loss_fun(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

val_loss /= len(val_loader.dataset)
val_accuracy = 100. * correct / total
print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%")


Validation Loss: 1.8197 | Validation Accuracy: 57.14%


## custom


In [45]:


custom_model.eval()  
val_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in val_loader2:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = loss_fun2(outputs, labels)

        val_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        correct += predicted.eq(labels).sum().item()
        total += labels.size(0)

val_loss /= len(val_loader.dataset)
val_accuracy = 100. * correct / total
print(f"Validation Loss: {val_loss:.4f} | Validation Accuracy: {val_accuracy:.2f}%")


Validation Loss: 2.6837 | Validation Accuracy: 28.57%


# Predicting


## pretrained


In [52]:
# Reload the model
reloaded_model = get_pretrained_resnet(num_classes)
reloaded_model.load_state_dict(torch.load("asl_model.pth"))
reloaded_model = reloaded_model.to(device)
reloaded_model.eval()

# ex inference on a single batch
for inputs, _ in val_loader:
    inputs = inputs.to(device)
    outputs = reloaded_model(inputs)
    _, predicted = outputs.max(1)
    print(f"Predicted labels: {predicted}")
    break




/Users/domingojr/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/domingojr/miniforge3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicted labels: tensor([18, 13,  8,  1,  8, 11,  6,  8, 12,  4,  6,  9,  8, 15,  9, 15, 16, 10,
        18, 17, 15, 15, 15,  0,  5, 15,  1,  5])


## custom


In [53]:
custom_loaded = load_model(ASLCNN, 'asl_model_custom.pth', hp.NUM_CLASSES)
custom_loaded.eval()
for inputs, _ in val_loader2:
    inputs = inputs.to(device)
    outputs = custom_loaded(inputs)
    _, predicted = outputs.max(1)
    print(f"Predicted labels: {predicted}")
    break



RuntimeError: mat1 and mat2 shapes cannot be multiplied (28x262144 and 65536x128)

# USING ON NEW IMAGES


In [48]:
%pip install pillow
%pip show pillow

Note: you may need to restart the kernel to use updated packages.
Name: pillow
Version: 10.4.0
Summary: Python Imaging Library (Fork)
Home-page: https://python-pillow.org
Author: 
Author-email: "Jeffrey A. Clark" <aclark@aclark.net>
License: HPND
Location: /Users/domingojr/miniforge3/lib/python3.12/site-packages
Requires: 
Required-by: matplotlib, torchvision
Note: you may need to restart the kernel to use updated packages.


In [82]:
image_path = "/Users/domingojr/Desktop/CS1430_Projects/final-project-Domingo-v/CNN_CODE_2 /A1297 copy.jpg"

## pretrained


In [84]:
reloaded_model = get_pretrained_resnet(num_classes)
reloaded_model.load_state_dict(torch.load("asl_model.pth"))
reloaded_model = reloaded_model.to(device)
reloaded_model.eval()

transform = get_transforms()

image = Image.open(image_path).convert("RGB") 
image = transform(image).unsqueeze(0)  # apply transformations with batch dimension (1, C, H, W)

input_tensor = image.to(device)

# classify the image
with torch.no_grad():
    output = reloaded_model(input_tensor)  # forward 
    _, predicted_class = torch.max(output, 1)  # class index

print(f"Predicted class index: {predicted_class.item()}")

train_dir = hp.TEST_DIR
train_dataset = datasets.ImageFolder(root=train_dir)

class_to_idx = train_dataset.class_to_idx
idx_to_class = {v: k for k, v in train_dataset.class_to_idx.items()}
print("Class to index:", class_to_idx)
print("PREDICTION: ",(idx_to_class.get(predicted_class.item())))



Predicted class index: 6
Class to index: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'nothing': 26, 'space': 27}
PREDICTION:  G


## custom


In [ ]:
num_classes = hp.NUM_CLASSES
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
testmodel = load_model(ASLCNN, "asl_model_custom.pth", num_classes)
testmodel = testmodel.to(device)
testmodel.eval()  # Set the model to evaluation mode

transform = get_transforms()

image = Image.open(image_path).convert("RGB") 
image = transform(image).unsqueeze(0)  # apply transformations with batch dimension (1, C, H, W)

input_tensor = image.to(device)

# classify the image
with torch.no_grad():
    output = testmodel(input_tensor)  # forward 
    _, predicted_class = torch.max(output, 1)  # class index

print(f"Predicted class index: {predicted_class.item()}")


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x262144 and 65536x128)